In [1]:
# Import packages
import pandas as pd
import json
import numpy as np

In [2]:
# Import datafiles
with open('PA_businesses.json', "r") as read_file:
    ratings = json.load(read_file) 
    
with open('PA_reviews_full.json', "r") as read_file:
    reviews = json.load(read_file)    

In [3]:
# Saving as dataframe 
df = pd.DataFrame(ratings['businesses']) 
df.head() 

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1RHY4K3BD22FK7Cfftn8Mg,Marathon Diner,"Center Core - Food Court, Fl 3, Pittsburgh Int...",Pittsburgh,PA,15231,40.496177,-80.246011,4.0,35,1,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Sandwiches, Salad, Restaurants, Burgers, Comfo...",None
1,dQj5DLZjeDK3KFysh1SYOQ,Apteka,4606 Penn Ave,Pittsburgh,PA,15224,40.465694,-79.949324,4.5,242,1,"{'CoatCheck': 'False', 'BusinessParking': '{'g...","Nightlife, Bars, Polish, Modern European, Rest...","{'Wednesday': '17:0-0:0', 'Thursday': '17:0-0:..."
2,v-scZMU6jhnmV955RSzGJw,No. 1 Sushi Sushi,436 Market St,Pittsburgh,PA,15222,40.441062,-80.002126,4.5,106,1,"{'OutdoorSeating': 'False', 'HasTV': 'True', '...","Japanese, Sushi Bars, Restaurants","{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'..."
3,KFbUQ-RR2UOV62Ep7WnXHw,Westwood Bar & Grill,825 Commonwealth Ave,West Mifflin,PA,15122,40.376674,-79.882480,3.0,5,1,"{'GoodForMeal': '{'dessert': False, 'latenight...","American (Traditional), Restaurants","{'Wednesday': '11:0-0:0', 'Thursday': '11:0-0:..."
4,QHr_fc_TLnGfUEuDEa2jDg,PA Wine & Spirits Store,"125 Towne Centre Dr, Ste 500",Wexford,PA,15090,40.632053,-80.056476,3.0,7,1,"{'BikeParking': 'False', 'RestaurantsTakeOut':...","Beer, Wine & Spirits, Food",None


In [ ]:
# Part 1, Question 1

In [4]:
def q1_yelp(data1,data2):
    
    #part one
    df = pd.DataFrame(data1['businesses']) #saving as dataframe 
    df.head() #looking at dataframe for understanding
    stars = df['stars'] #extracting stars and saving as pandas series 
    star_percentages = stars.value_counts()/len(stars)*100 #value counts gives you values gives you counts for each star 

    answer1 = pd.DataFrame(star_percentages).sort_index() #sorting the answers by index
    
    #part two
    reviews_df = pd.DataFrame(data2['reviews']) #extract the dataframe and save it to a more digestible form 
    reviews_df['text'] = reviews_df['text'].str.split(' ') #split reviews by white space to get array of words 
    reviews_df['word_count'] = reviews_df['text'].apply(lambda x: len(x)) #create new column for wordcount
    #for each row, count how many words (take length)
    answer2 = pd.DataFrame(reviews_df.groupby(['stars']).mean()['word_count'])
    
    # return answer1a, answer1b

In [5]:
answer1a, answer1b = q1_yelp(ratings,reviews)

NameError: name 'answer1a' is not defined

In [ ]:
q1_yelp(ratings,reviews)

In [ ]:
# Part 1, Question 2

In [ ]:
def q2_yelp(data1):
    
    #Part 1
    df = pd.DataFrame(data1['businesses'])
    df['categories'] = df['categories'].str.split(',').apply(lambda x: [i.strip().lower() for i in x]) 
    #Split categories by commas to get list of labels, remove white space at the start and end for all values, 
    #standardise so all the strings are in the same case

    s = df.apply(lambda x: pd.Series(x['categories']),axis=1).stack().reset_index(level=1, drop=True)
    s.name = 'label'
    #create a variable to count unique category lables
    #apply pd.Series to each row to turn list of categories into a big data frame,
    #with columns corresponding to each category
    #Stack into one big column of all labels and use set to identify unique 

    df_unstacked = df.drop('categories', axis=1).join(s)
    #remove category label from column and combine with s
  
    answer1 = len(set(df_unstacked['label']))
    
    #Part 2 & 3
    
    label_df = pd.DataFrame(list(zip(df_unstacked.groupby('label').mean()['stars'],df_unstacked.groupby('label').count()['business_id'])),columns = ['mean_stars','counts'])
    
    #create a dataframe that contains by use of zip two columns: label and mean stars with column names etc 
    

    label_df.index = df_unstacked.groupby('label').mean()['stars'].index
    #for some reason index dispappeated
    #relabel index using label names 
    
    answer2 = label_df
    
    # return answer1,answer2

In [ ]:
answer2a, answer2bc = q2_yelp(ratings)

In [ ]:
# Part 1, Question 3

In [ ]:
def q3_yelp(data): 
    df = pd.DataFrame(data['businesses'])

    high_ratings = df[['attributes','stars']]
    #create dataframe with attributes and stars 
    
    dataframe = pd.DataFrame() #create empty dataframe
    
    for i in range(len(ratings['businesses'])): 
        dataframe = dataframe.append(pd.DataFrame(ratings['businesses'][i]['attributes'],index=[i]))
        
        #for each rating in the data set, extract attributes and turn them into a nicely formatted column
        
    dataframe['stars'] = df[['stars']]
    
    #bring stars data into our nicely formatted dataframe 
    
    dataframe = dataframe[dataframe['stars']>3.5] #take values that are 4 or higher 
    
    descriptive = dataframe.drop(['stars'],axis=1).describe() #get rid of the stars 
    #because we want summary statistics only of the attributes 
    
    descriptive_keys = (descriptive.loc['count'][descriptive.loc['count']>500]).keys()
    #take names of attributes of which we have more than 500 ratings because some ratings only exists once or twice
    
    ratings_filtered = dataframe[descriptive_keys]
    
    for i in ratings_filtered.columns:
        print(i.upper())
        print(ratings_filtered[i].value_counts() )
        print()
        
    #print descriptive statistics for each attribute 

In [ ]:
# Part 2

In [6]:
# wiki_api.py
import hashlib, os, json, requests
from lxml import etree

# Input: Page name of a Wikipedia article.
# Returns: Full HTML source of the named article if it exists,
#          or None if no such page exists.
def __api_GET_latest_page(title):
    parameters = {
        "action": "parse",
        "page": title,
        "format": "json"
    }
    response_json = __get("revisions", title, parameters)
    if("parse" in response_json.keys() 
        and "text" in response_json["parse"].keys() 
        and "*" in response_json["parse"]["text"].keys()):
        return response_json["parse"]["text"]["*"]
    return None    

# Internal function to hide a caching API request into a single private function.
# This function will save you a lot of headaches in writing your own HTTP requests
# and will save the Wikimedia foundation some bandwidth since you'll fetch a local
# copy if you have already retrieved an article text at least once.
def __get(function_key, key, parameters, check_cache=True, write_cache=True):
    target = "https://en.wikipedia.org/w/api.php"
    cache_path = "cached_api"
    params_unicode = str(parameters).encode('utf-8')
    md5 = hashlib.md5(params_unicode).hexdigest()
    return_json = None

    cache_file = os.path.join(cache_path, function_key, str(key), md5)
    cache_exists = os.path.isfile(cache_file)
    if cache_exists:
        try:
            json_in = open(cache_file, "r")
            json_str = json_in.read()
            return_json = json.loads(json_str)
            if "error" in return_json.keys() and "code" in return_json["error"].keys() and return_json["error"]["code"]=="maxlag":
                cache_exists = False
        except:
            cache_exists = False

    if not cache_exists:
        cache_dir = os.path.dirname(cache_file)
        if not os.path.isdir(cache_dir):
            os.makedirs(cache_dir)
        r = requests.get(target, params=parameters)
        request_json = r.json()
        json_out = open(cache_file, "w")
        print(json.dumps(request_json), file=json_out)
        return_json = request_json
    return return_json

# This function takes as input a parsed HTML tree and returns the same
# tree but with a set of tags removed, mostly the contents of tables and scripts.
# This makes parsing the actual contents of a page easier.
def __remove_tables_and_scripts(tree):
    tags_to_remove = ["tbody", "td", "script"]
    for tag in tags_to_remove:
        elements = tree.find(f".//{tag}")
        if elements is not None:
            for e in elements:
                e.getparent().remove(e)
    return tree

# This function takes two required and one optional parameters as input.
#
# Required:
# name: Name of a Wikipedia page to retrieve.
# format: Type of content that you want returned. Options include:
#         "html" : Full HTML content of the page you requested.
#         "text" : Full content of the page you requested as a single string,
#                  with all HTML tags removed.
#         "list" : Full content of the page you requested with all HTML removed,
#                  but each paragraph on the page is a separate string, and the
#                  page as a whole is returned to you as a list of paragraphs.
#
# Optional:
# include_tables: By default, all tables and scripts in the HTML text will be 
#                 removed from the text that gets sent back to you. If you want
#                 to include that content, you can pass in True instead.
#
# This function returns the content of the page in the format that you specified.
def page_text(name, format, include_tables = False):
    try:
        result = __api_GET_latest_page(name)
    except:
        print("API request failed.")
    if result:
        e = etree.fromstring(result)
        if not include_tables:
            e = __remove_tables_and_scripts(e)
        if format == "html":
            return str(etree.tostring(e))
        elif format == "text":
            return ''.join(e.itertext())
        elif format == "list":
            return ''.join(e.itertext()).split('\n')
    else:
        print("Failed to retrieve a page.")
        return None

In [ ]:
from wiki_api import page_text
